# Hands-on: NLP for Social Science

Material preparado para [SICSS Chile 2025](https://sicss.io/2024/chile/). Síentete libre de utilizarlo, modificarlo y compartirlo.

**Autor**: Jorge Ortiz Fuentes

## Objetivo

Explorar técnicas de procesamiento de lenguaje natural para obtener insights en ciencias sociales.

## Contenidos

Los contenidos de la clase son:
* Cargar y explorar un corpus.
* Preprocesar textos (limpieza, tokenización y lematización).
* Explorar textos a nivel de palabras (palabras más frecuentes y n-gramas).
* Representar textos en forma de vectores (bag-of-words, TF-IDF, embeddings).
* Extraer tópicos de los textos (LDA y BERTopic).
* Etiquetar textos con categorías gramáticales (POS tagging) y entidades nombradas (NER).
* Clasificar textos usando modelos Transformer preentrenados desde HuggingFace.
* Análisis de textos usando IA generativa con Langchain.
* Entrenar un clasificador de textos usando embeddings, scikit-learn y xgboost.

Para ello, vamos a trabajar con un corpus de 10.000 noticias chilenas del periodo comprendido entre octubre y diciembre de 2019. El corpus se encuentra en el archivo `noticias_oct_dic_2019.tsv` y contiene las siguientes columnas:

* `texto`: texto de la noticia.
* `canal`: medio de comunicación que publicó la noticia.
* `fecha`: fecha de publicación de la noticia.

En esta clase trabajaremos con las librerías `pandas`, `spaCy`, `scikit-learn`, `transformers`, `gensim`, `langchain`, `spanish-nlp` y `bertopic` y `sentence-transformers`.



## Carga y exploración de textos

Las noticias se encuentran en un archivo `tsv` (tab-separated values), que es un formato de archivo de texto plano que utiliza tabuladores para separar los campos. 

In [ ]:
# Descargar el archivo de noticias

!wget https://raw.githubusercontent.com/jorgeortizfuentes/sicss-text-analysis2025/refs/heads/main/noticias_oct_dic_2019.tsv

In [ ]:
import pandas as pd

df = pd.read_csv("noticias_oct_dic_2019.tsv", sep="\t")

df

Para ver cuántas noticias hay por medio en el corpus, podemos usar el método `value_counts()` de `pandas`:

In [ ]:
df["medio"].value_counts()

Vamos a ver un texto de ejemplo:

In [ ]:
print("Texto")
print(df["texto"].iloc[100])

Si se desea contar la cantidad de caracteres de un texto se puede ocupar la función `len()` de Python:

In [ ]:
ejemplo = "soy un texto de prueba de 39 caracteres"
len(ejemplo)

A continuación, vamos a contar cuántos caracteres posee cada noticia. Para ello, podemos definir una función que cuente los caracteres de un texto y luego aplicarla a cada noticia usando el método `apply()` de `pandas`:

In [ ]:
def contar_caracteres(texto):
    return len(texto)

df["caracteres"] = df["texto"].apply(contar_caracteres)

df

Luego, podemos obtener el promedio de caracteres por noticia, la desviación estándar, el mínimo y el máximo, y los cuartiles usando el método `describe()` de `pandas`:

In [ ]:
df["caracteres"].describe()

Podemos ver que la cantidad mínima de caracteres, la máxima y los percentiles 25, 50 y 75. 

Vemos que el mínimo y el máximo se alejan significativamente de la media de caracteres. 

Estos datos se consideran `outliers`. Para ver cuáles son los noticias más cortos y más largos, podemos ordenar el `DataFrame` por la columna `caracteres` usando el método `sort_values()` de `pandas`:



In [ ]:
df.sort_values(by="caracteres")

Si deseamos ver el texto completo de una fila, podemos usar el método `iloc[]` de `pandas` para acceder a una fila en particular a partir de su índice. Por ejemplo, para ver el texto más largo:

In [ ]:
print(df.iloc[1490]["texto"])

Vamos a eliminar los `outliers` del corpus. Para ello, vamos a utilizar el Rango Intercuartil (IQR), que corresponde a la diferencia entre el tercer y el primer cuartil. 


![IQR](https://github.com/jorgeortizfuentes/sicss-text-analysis2025/blob/main/images/iqr.jpg?raw=true)



Luego, vamos a eliminar las noticias que se encuentren fuera del rango $[Q_1 - 1.5 \times IQR]$ (valores menores a este rango) y $[Q_3 + 1.5 \times IQR]$ (valores mayores a este rango).




In [ ]:
# Calculamos el IQR
q3 = df["caracteres"].quantile(0.75)
q1 = df["caracteres"].quantile(0.25)
iqr = q3 - q1

iqr1_5_menor = q1 - 1.5 * iqr
iqr1_5_mayor = q3 + 1.5 * iqr

print(iqr1_5_menor)
print(iqr1_5_mayor)

Vamos entonces a eliminar los `outliers` del corpus. Esto es particularmente útil cuando se tienen datos con problemas, por ejemplo porque quedaron mal descargados.

La cota inferior corresponde a un número negativo, lo que no tiene sentido. Por lo tanto, vamos a definir arbitrariamente el umbral menor como 100 caracteres.

In [ ]:
custom_lower_bound = 100

df_filtrado = df[(df["caracteres"] > custom_lower_bound) & (df["caracteres"] < iqr1_5_mayor)]

# Reseteamos el índice
df_filtrado = df_filtrado.reset_index(drop=True)

df_filtrado

In [ ]:
df_filtrado["caracteres"].describe()

## Preprocesamiento de textos

Un paquete muy útil para el procesamiento de textos es `spaCy`, que permite realizar diversas tareas de procesamiento de lenguaje natural. Para instalarlo, se puede ejecutar el siguiente comando en la consola:

### Instalación de spaCy

In [ ]:
!pip install spacy

Luego podemos descargar la versión en español del modelo `spaCy`:

In [ ]:
!python -m spacy download es_core_news_sm

Y podemos cargar el modelo:

In [14]:
import spacy

nlp = spacy.load("es_core_news_sm")

### Tokenización

Para trabajar con los textos, necesitamos separarlos en unidades de análisis. Este proceso se conoce como `tokenización`, pues se separa el texto en `tokens`. En este caso, vamos a separar los textos en palabras o símbolos de puntuación. 

Seleccionemos un texto al azar para ver cómo funciona la tokenización de `spaCy`:

In [ ]:
texto_prueba = df_filtrado.iloc[100]["texto"]
texto_tokenizado = [token.text for token in nlp(texto_prueba)]
texto_tokenizado

### Lematización

Se conoce como `lematización` al proceso de reducir las palabras a su forma base o `lema`. Por ejemplo, el lema de las palabras `corriendo`, `correr` y `corrió` es `correr`.

Para lematizar un texto, podemos usar el atributo `lemma_` de cada token:

In [ ]:
texto1 = "los perros corren"
texto2 = "el perro corrió"

lemas_texto1 = [token.lemma_ for token in nlp(texto1)]
lemas_texto2 = [token.lemma_ for token in nlp(texto2)]

print(lemas_texto1)
print(lemas_texto2)

### Eliminación de stopwords

Se conocen como `stopwords` a las palabras cuyo significado léxico no es relevante para un análisis a nivel palabra. Por ejemplo, las palabras como las preposiciones, artículos y conjunciones no aportan información relevante para un análisis a nivel palabra.

Vamos a utilizar la lista de `stopwords` en español de `spaCy`:

In [ ]:
spacy.lang.es.stop_words.STOP_WORDS

In [ ]:
texto_prueba = df_filtrado.iloc[100]["texto"]
print(texto_prueba)
texto_tokenizado = [
    token.lemma_ for token in nlp(texto_prueba) if not token.is_stop and not token.is_punct
]
texto_tokenizado

## Preprocesamiento adicional

Muchas veces es necesario realizar un preprocesamiento adicional para eliminar caracteres especiales, URLs, números y para normalizar palabras. Por ejemplo, en los textos encontramos en múltiples ocasiones la palabra  `José` y `Jose`, que corresponden a la misma palabra pero escrita de forma distinta.

Para esto podemos ocupar el paquete `spanish_nlp`

In [ ]:
!pip install spanish-nlp
!pip install unidecode

In [ ]:
from spanish_nlp import SpanishPreprocess

sp = SpanishPreprocess(
    lower=True,
    remove_url=True,
    remove_hashtags=False,
    split_hashtags=True,
    normalize_breaklines=True,
    remove_emoticons=False,
    remove_emojis=False,
    convert_emoticons=False,
    convert_emojis=False,
    normalize_inclusive_language=True,
    reduce_spam=True,
    remove_vowels_accents=True,
    remove_multiple_spaces=True,
    remove_punctuation=True,
    remove_unprintable=True,
    remove_numbers=True,
    remove_stopwords=False,
    stopwords_list=None,
    lemmatize=False,
    stem=False,
    remove_html_tags=True,
)

test_text = """𝓣𝓮𝔁𝓽𝓸 𝓭𝓮 𝓹𝓻𝓾𝓮𝓫𝓪

<b>Holaaaaaaaa a todxs </b>, este es un texto de prueba :) a continuación les mostraré un poema de Roberto Bolaño llamado "Los perros románticos" 🤭👀😅

https://www.poesi.as/rb9301.htm

¡Me gustan los pingüinos! Sí, los PINGÜINOS 🐧🐧🐧 🐧 #VivanLosPinguinos #SíSeñor #PinguinosDelMundoUníos #ÑanduesDelMundoTambién

Si colaboras con este repositorio te puedes ganar $100.000 (en dinero falso). O tal vez 20 pingüinos. Mi teléfono es +561212121212"""

print(sp.transform(test_text, debug=False))

Vamos a aplicar este preprocesamiento a los textos y guardarlos en una nueva columna:

In [22]:
df_filtrado["texto_pp"] = df_filtrado["texto"].apply(lambda x: sp.transform(x))

## Exploración a nivel de palabras

Ahora, vamos a explorar el corpus a nivel de palabras. Para ello, vamos a crear una función que reciba un texto y devuelva una lista con las palabras tokenizadas, en minúsculas, lematizadas, sin `stopwords` y sin números.

In [ ]:
def tokenizar(text):
    doc = nlp(text)
    return [
        token.lemma_ for token in doc if not token.is_punct and not token.is_stop and token.is_alpha
    ]


df_filtrado["tokens"] = df_filtrado["texto_pp"].apply(tokenizar)
df_filtrado

Vamos a utilizar la función `Counter` de la librería `collections` para contar la frecuencia de las palabras en el corpus. Luego, vamos a mostrar las 30 palabras más frecuentes.

In [ ]:
from collections import Counter

# Unir todos los tokens en una lista
all_tokens = [token for tokens in df_filtrado["tokens"] for token in tokens]

# Contar la frecuencia de las palabras
word_freq = Counter(all_tokens)

# Obtener las 20 palabras más frecuentes
top_words = word_freq.most_common(30)

print(top_words)

### N-gramas más comunes

Los n-gramas son secuencias de n palabras. Por ejemplo, los bi-gramas de la oración `El perro juega con su pelota` son:
- `El perro`
- `perro juega`
- `juega con`
- `con su`
- `su pelota`

Los n-gramas son útiles para detectar frases o expresiones que se repiten en el corpus. Para ello, vamos a utilizar `CountVectorizer` de `scikit-learn`:



In [25]:
from sklearn.feature_extraction.text import CountVectorizer

stopwords = list(spacy.lang.es.stop_words.STOP_WORDS)

# Crear instancia de CountVectorizer
vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words=stopwords)

Luego, vamos a utilizar el método `fit_transform` de la instancia de `CountVectorizer` para crear una matriz de conteo de n-gramas.



In [ ]:
# Crear matriz de conteo de n-gramas
ngram_matrix = vectorizer.fit_transform(df_filtrado["texto_pp"])

ngram_matrix

Finalmente, vamos a sumar las frecuencias de los n-gramas y a mostrar los n-gramas más comunes.

In [ ]:
import numpy as np

# Sumar las frecuencias de los n-gramas
ngram_freq = np.sum(ngram_matrix, axis=0)

# Obtener los n-gramas más comunes
top_ngrams = [(ngram, ngram_freq[0, index]) for ngram, index in vectorizer.vocabulary_.items()]
top_ngrams = sorted(top_ngrams, key=lambda x: x[1], reverse=True)[:30]

print(top_ngrams)

Calculemos ahora las palabras y los bigramas más frecuentes por medio. Para ello concatenaremos todos los textos de cada medio y luego calcularemos sus n_gramas más repetidas. 

In [ ]:
medios = df_filtrado["medio"].unique().tolist()

vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

# Iterar sobre los medios
for medio in medios:
    # Filtrar los textos por medio
    textos_year = df_filtrado[df_filtrado["medio"] == medio]["texto_pp"]

    # Unir todos los textos en un solo string
    texto_completo = " ".join(textos_year)

    # Crear la matriz de n-gramas
    ngram_matrix_year = vectorizer.fit_transform([texto_completo])

    # Sumar las frecuencias de los n-gramas
    ngram_freq_year = np.sum(ngram_matrix_year, axis=0)

    # Obtener los n-gramas y sus frecuencias
    top_10_ngrams_year = [
        (ngram, ngram_freq_year[0, index]) for ngram, index in vectorizer.vocabulary_.items()
    ]

    # Ordenar por frecuencia y tomar los 10 más frecuentes
    top_10_ngrams_year = sorted(top_10_ngrams_year, key=lambda x: x[1], reverse=True)[:10]

    # Imprimir los resultados para el medio actual
    print(f"Medio: {medio}")
    print(top_10_ngrams_year)
    print("\n")

## Vectorización de textos

Para poder hacer análisis más profundos de los textos, necesitamos representarlos en forma numérica. Este proceso se conoce como `vectorización`, es decir, transformar los textos en vectores (secuencias de números).

Existen diversas formas de vectorizar textos. En esta clase vamos a ver dos de ellas: `bag-of-words` y `TF-IDF`.

### Bag of words

Se conoce como `bag-of-words` a la representación de un texto como un vector que contiene la frecuencia de cada palabra en el texto. Es decir, se crea un vector con todas las palabras del corpus y se cuenta cuántas veces aparece cada palabra en cada texto.

![Bag of ords](https://vitalflux.com/wp-content/uploads/2021/08/Bag-of-words-technique-to-convert-to-numerical-feature-vector-png.png)

También, podemos contar la frecuencia de los n-gramas en cada texto. Para ello, nuevamente, vamos a utilizar `CountVectorizer` de `scikit-learn`. Esta vez con el parámetro `ngram_range` (1, 1) (por defecto) para contar palabras y con el parámetro `ngram_range` (1, 2) para contar palabras y 2-gramas.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_by_word = CountVectorizer(ngram_range=(1, 1), stop_words=stopwords)

# Entrenamos el vectorizador
vectorizer_by_word.fit(df_filtrado["texto_pp"])

# Lo aplicamos al texto de ejemplo
vectorizer_by_word.transform([texto_prueba])

El resultado del texto es un vector muy  largo, que corresponde a la cantidad de palabras únicas en el corpus.

Ahora, vamos a vectorizar un texto con `ngram_range` (1, 2) para contar 

In [ ]:
vectorizer_by_2gram = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

# Entrenamos el vectorizador
vectorizer_by_2gram.fit(df_filtrado["texto_pp"])

# Lo aplicamos al texto de ejemplo
vectorizer_by_2gram.transform([texto_prueba])

Ahora tenemos una matriz aún más grande que la anterior. Esto se debe a que ahora estamos contando palabras y 2-gramas.

Una dimensionalidad muy alta puede ser un problema, debido a que es más costoso computacionalmente de procesar. Por lo tanto, vamos a limitar la cantidad de palabras que se van a considerar en la vectorización. Para ello, vamos a utilizar el parámetro `max_features` de `CountVectorizer` para limitar la cantidad de palabras y n-gramas a los 1000 más frecuentes

In [ ]:
vectorizer_by_2gram = CountVectorizer(ngram_range=(1, 2), max_features=1000, stop_words=stopwords)

# Entrenamos el vectorizador
vectorizer_by_2gram.fit(df_filtrado["texto_pp"])

# Lo aplicamos al texto de ejemplo
vectorizer_by_2gram.transform([texto_prueba])

Ahora aplicamos la vectorización a todos los textos del corpus y creamos una matriz de 1000 columnas y tantas filas como textos haya en el corpus.

In [ ]:
corpus_vectorizado_bow = vectorizer_by_2gram.transform(df_filtrado["texto_pp"])

# Lo convertimos a un DataFrame con los nombres de las columnas
df_corpus_vectorizado_bow = pd.DataFrame(
    corpus_vectorizado_bow.toarray(), columns=vectorizer_by_2gram.get_feature_names_out()
)
df_corpus_vectorizado_bow

Aunque puede parecer una vectorización sencilla y poco sofisticada, `bag-of-words` es una técnica bastante útil ya que logra capturar el significado de los textos. Por ejemplo, si tenemos dos textos que hablan de fútbol, es probable que compartan palabras como `gol`, `equipo`, `jugador`, `partido`, etc. Por lo tanto, es probable que los vectores de estos textos sean similares. Por ejemplo, podemos buscar el texto que más se parece a `texto_prueba`. 

Para ello recorremos todos los textos del corpus y calculamos la similitud `coseno` con el texto de prueba. La similitud coseno es una medida de similitud entre dos vectores que mide el coseno del ángulo entre ellos. Si los vectores son idénticos, la similitud coseno es 1. Si los vectores son ortogonales, la similitud coseno es 0. Si los vectores son opuestos, la similitud coseno es -1.

Para calcular la similitud coseno, vamos a utilizar la función `cosine_similarity` de `scikit-learn`:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Hay que calcular la similitud entre el texto de prueba y todos los demás.
matriz_similitud = cosine_similarity(corpus_vectorizado_bow[100], corpus_vectorizado_bow)

# Ordenar los textos por similitud
textos_similares = list(zip(df_filtrado["texto_pp"], matriz_similitud[0]))
textos_similares.sort(key=lambda x: -x[1])

# Mostrar los 5 textos más similares
for texto, similitud in textos_similares[:6]:
    print(f"Similitud: {similitud}\n{texto}\n")

### TF-IDF

Aunque `bag-of-words` puede ser útil, no logra capturar la importancia de las palabras, ya que solo cuenta su frecuencia. Para ello, se puede usar `TF-IDF` (Term Frequency - Inverse Doucument Frequency), que cuenta la frecuencia de las palabras ponderada por su frecuencia inversa en los textos. Es decir, si una palabra aparece en muchos textos, no aporta mucha información para distinguir entre textos, por lo tanto se le asigna una ponderación menor.

Para aplicar `TF-IDF` sobre el corpus, vamos a utilizar la función `TfidfVectorizer` de `scikit-learn`.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creamos la instancia de TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000, stop_words=stopwords)

# Entrenamos el vectorizador y transformamos el texto
# (lo hacemos en un solo paso con fit_transform)
corpus_vectorizado_tfidf = vectorizer.fit_transform(df_filtrado["texto_pp"])

# convertir a DataFrame
df_corpus_vectorizado_tfidf = pd.DataFrame(
    corpus_vectorizado_tfidf.toarray(), columns=vectorizer.get_feature_names_out()
)

# Calculamos la similitud del texto de prueba con los demás
matriz_similitud_tfidf = cosine_similarity(corpus_vectorizado_tfidf[100], corpus_vectorizado_tfidf)

# Ordenamos los textos por similitud
textos_similares_tfidf = list(zip(df_filtrado["texto_pp"], matriz_similitud_tfidf[0]))
textos_similares_tfidf.sort(key=lambda x: -x[1])

# Mostramos los 5 textos más similares
for texto, similitud in textos_similares_tfidf[:5]:
    print(f"Similitud: {similitud}\n{texto}\n")

## Extraer tópicos de los textos

Cuando tenemos muchos textos, es útil poder extraer los tópicos de los textos, es decir, las temáticas que abordan los textos. Para ello, vamos a utilizar dos métodos.

Para ello ocuparemos `Latent Dirichlet Allocation` (LDA), que es un modelo generativo de tópicos. Este modelo asume que cada texto se compone de una mezcla de tópicos y que cada tópico se compone de una mezcla de palabras.


## LDA con TF-IDF

Vamos a aplicar LDA a la matriz generada por `tf-idf`. Si bien también podríamos ocupar Bag of Words, TF-IDF es más adecuado para LDA, ya que pondera las palabras por su importancia en los textos.

Creamos la instancia de LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, random_state=0)

lda

In [ ]:
# Entrenamos LDA
lda.fit(corpus_vectorizado_tfidf)

In [ ]:
# Podemos obtener los tópicos con el método `components_` de la instancia de LDA:

topicos = lda.components_
topicos

Tenemos una matriz de 10 tópicos y 1000 palabras.

Vamos a mostrar las 20 palabras que componen cada tópico:

In [ ]:
# Obtener las palabras de cada tópico
words = vectorizer_by_2gram.get_feature_names_out()

top_words_per_topic = []

for numero_topico in range(10):
    print(f"\nTópico {numero_topico + 1}")
    print("-" * 50)

    # Obtener los pesos de las palabras para este tópico
    pesos_palabras = topicos[numero_topico]

    # Crear lista de tuplas (palabra, peso)
    palabras_y_pesos = list(zip(words, pesos_palabras))

    # Ordenar por peso de mayor a menor
    palabras_y_pesos = sorted(palabras_y_pesos, key=lambda x: x[1], reverse=True)

    # Mostrar las 20 primeras palabras
    for palabra, peso in palabras_y_pesos[:10]:
        print(f"Palabra: {palabra:<20} Peso: {peso:.4f}")

## Extracción de Tópicos con BERTopic

LDA (Latent Dirichlet Allocation) es un método popular para la extracción de tópicos, pero presenta algunas limitaciones:

- LDA supone que los documentos se generan a partir de una mezcla de tópicos y que cada tópico es una mezcla de palabras, lo cual puede no ser cierto en todos los casos.
- LDA no siempre maneja bien documentos cortos o con tópicos poco definidos.

### BERTopic: un enfoque más moderno

BERTopic utiliza modelos de lenguaje basados en `Transformers` (como BERT) para generar representaciones vectoriales de textos (`embeddings`). Luego, agrupa estos vectores para identificar tópicos, lo que ofrece ventajas sobre LDA:
- No asume una distribución específica de palabras en los tópicos.
- Capta mejor el contexto semántico de los textos.
- Es más robusto con textos cortos


### Instalación de BERTopic

Para comenzar, instalamos la librería con el siguiente comando:


In [ ]:
!pip install bertopic

Importamos la librería y creamos una instancia de BERTopic. Utilizaremos el modelo multilingüe `paraphrase-multilingual-MiniLM-L12-v2`, que soporta español:

In [44]:
from bertopic import BERTopic

# Crear una instancia del modelo BERTopic
topic_model = BERTopic(
    language="multilingual", embedding_model="paraphrase-multilingual-MiniLM-L12-v2", verbose=True
)

Entrenamos el modelo con los textos originales. No es necesario preprocesar los textos, ya que los modelos de lenguaje basados en Transformers manejan bien los textos en bruto.

El resultado será un DataFrame que incluye:
- ID del tópico: Identificador único.
- Frecuencia: Número de textos asignados al tópico.
- Palabras representativas: Términos más importantes del tópico.
- Tópico -1: Corresponde a los textos que no pudieron asignarse a ningún tópico (outliers).


In [ ]:
# Entrenar el modelo y obtener los tópicos
topics, probs = topic_model.fit_transform(df_filtrado["texto"])

# Obtener información detallada de los tópicos
topic_info = topic_model.get_topic_info()
topic_info

In [ ]:
# Asignar tópicos a los textos
df_filtrado["topico_bertopic"] = topics
df_filtrado

In [ ]:
# Obtener el tópico más frecuente por medio


for medio in sorted(df_filtrado["medio"].unique()):
    # Filter comments for this year
    noticias_medio = df_filtrado[df_filtrado["medio"] == medio]

    # Filter comments with no topic (-1)
    noticias_medio = noticias_medio[noticias_medio["topico_bertopic"] != -1]

    # Count frequency of each topic
    frecuencia_topicos = noticias_medio["topico_bertopic"].value_counts()

    # Get most frequent topic
    topico_mas_frecuente = frecuencia_topicos.idxmax()

    # Get topic info for the most frequent topic
    info_topico = topic_model.get_topic(topico_mas_frecuente)

    print(f"\Medio: {medio}")
    print(f"Tópico más frecuente: {topico_mas_frecuente}")
    print(f"Frecuencia: {frecuencia_topicos[topico_mas_frecuente]}")
    print("Palabras más representativas:")
    for palabra, peso in info_topico[:10]:
        print(f"- {palabra}: {peso:.4f}")

## Análisis lingüísticos

A continuación vamos a etiquetar textos mediante análisis lingüísticos. En concreto, vamos a realizar las siguientes tareas:

* Análisis de categorías gramáticales (POS tagging)
* Análisis de entidades nombradas (NER)

En concreto, vamos a utilizar Spacy para realizar el etiquetado de tokens.  

In [ ]:
# Para etiquetar un texto con categorías gramáticas, podemos usar el atributo `pos_` de cada token. Este atributo etiqueta las palabras con categorías universales, como Sustantivo, Verbo, Adjetivo, etc.
texto_prueba = df_filtrado.iloc[13]["texto"]
print(texto_prueba)
pos_texto = [(token.text, token.pos_) for token in nlp(texto_prueba)]
pos_texto

In [ ]:
# Para identificar las entidades nombradas, podemos usar el atributo `ents` del resultado de procesar un texto con `nlp()`. Este atributo retorna una lista de entidades, cada una con su tipo (como Persona, Lugar, Organización, etc) y el texto de la entidad.

texto_prueba = df_filtrado.iloc[20]["texto"]
print(texto_prueba)
entidades = [(ent.text, ent.label_) for ent in nlp(texto_prueba).ents]
entidades

## Clasificación de textos

En muchas ocasiones, es necesario clasificar textos en distintas categorías. 

## Modelos preentrenados con HuggingFace

HuggingFace es una librería que permite acceder a modelos preentrenados de lenguaje natural. En particular, vamos a utilizar modelos preentrenados para clasificar textos en las siguientes categorías:

- Análisis de sentimientos
- Hate speech

Puedes encontrar más información sobre los modelos preentrenados en español de HuggingFace en [este enlace](https://huggingface.co/models?pipeline_tag=text-classification&language=es&sort=trending).

Primero vamos a instalar la librería `transformers` de HuggingFace:

In [ ]:
!pip install transformers

Ahora, vamos a utilizar el modelo `pysentimiento/robertuito-sentiment-analysis` para clasificar los textos dependiendo de su sentimiento. Este modelo clasifica los textos en tres categorías: `positivo`, `negativo` y `neutro`.

In [ ]:
from transformers import pipeline

# Crea una instancia del pipeline de análisis de sentimiento
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis",
    return_all_scores=True,
)

# Texto de prueba
texto_prueba = "la constitución es un mamarracho"

sentiment_analyzer(texto_prueba)

Ahora vamos a utilizar el modelo `pysentimiento/robertuito-hate-speech` para clasificar los textos dependiendo de si contienen discurso de odio o no. Este modelo clasifica los textos en las siguientes clases:
- HS: si el texto contiene discurso de odio.
- TR: si el texto está dirigido a una persona específica.
- AG: si el texto es agresivo.

In [ ]:
# Crea una instancia del pipeline de análisis de hate
hate_analyzer = pipeline(
    "text-classification",
    model="pysentimiento/robertuito-hate-speech",
    return_all_scores=True,
)

# Texto de prueba
texto_prueba = "los constituyentes son unos estupidos"

hate_analyzer(texto_prueba)

Procesemos ahora todo el dataset con el modelo de análisis de sentimientos y de hate speech.

In [ ]:
# df_filtrado["sentimiento"] = df_filtrado["texto"].apply(lambda x: sentiment_analyzer(x))
# df_filtrado["hate_speech"] = df_filtrado["texto"].apply(lambda x: hate_analyzer(x))

## Análisis de textos con IA generativa

La IA generativa ha ganado popularidad en los últimos años para el análisis de textos ya que no requiere de datos preetiquetados. Basta con introducir las instrucciones necesarias para que el modelo genere clasificaciones, análisis, resúmenes, traducciones o lo que se necesite.

Existen múltiples modelos de IA generativa, como ChatGPT, Gemini, Llama, Claude, entre otros. 

En esta clase vamos a utilizar Langchain, un paquete que nos permite trabajar con IA generativa de forma agnóstica a los modelos. 

A continuación, ocuparemos Gemini, la IA generativa de Google, para detectar las noticias que contienen sesgos. 

Primero, vamos a instalar las dependencias necesarias:

In [ ]:
!pip install langchain
!pip install langchain-openai

Para ocupar los modelos de IA generativa necesitas una API Key del proveedor. En este caso, usaremos la API de ChatGPT.

Para obtener la API Key, debes registrarte en OpenAI, ingresar tu tarjeta de crédito y generar la API Key.
    
https://platform.openai.com/docs/overview

Considera que el uso de estos servicios tiene un costo asociado. Puedes ver el detalle de los precios en la siguiente página:
    
https://openai.com/api/pricing/

In [136]:
import os

os.environ["OPENAI_API_KEY"] = (
    "xxx"
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Grado de creatividad. 0 significa que es poco creativo y 1 es muy creativo
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

llm.invoke("Saludame en mapudungun")

In [140]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Analiza la siguiente noticia y determina si el contenido o la forma de estar escrito puede generar temor en el lector.

Entenderemos como temor la sensación de miedo, angustia o preocupación que puede generar un texto en el lector.

Solo extrae las propiedades mencionadas en la función 'Classification'.

Texto:
{texto}
"""
)


class Classification(BaseModel):
    razonamiento: str = Field(description="El razonamiento detrás de la clasificación")
    temor: bool = Field(description="Si el texto genera o no temor")


chain = tagging_prompt | llm.with_structured_output(Classification)

In [ ]:
noticia = df["texto"].iloc[2]

resultado = chain.invoke({"texto": noticia})
print("Noticia:")
print(noticia)
print("\n\nRazonamiento:")
print(resultado.razonamiento)
print("\n\nGenera temor:", resultado.temor)

In [112]:
#df_filtrado_sample = df_filtrado.sample(200, random_state=0)

In [142]:
def aplicar_chain(texto):
    resultado = chain.invoke({"texto": texto})
    return resultado

In [ ]:
# # Crea 3 nuevas columnas en el DataFrame con los resultados de la clasificación
# df_filtrado["clasificacion_estereotipo"] = df_filtrado["texto"].apply(aplicar_chain)

# df_filtrado["razonamiento"] = df_filtrado["clasificacion_estereotipo"].apply(
#     lambda x: x.razonamiento
# )
# df_filtrado["genera_temor"] = df_filtrado["clasificacion_estereotipo"].apply(lambda x: x.temor)
# df_filtrado

In [144]:
# df_filtrado.to_csv("noticias_procesadas.tsv", sep="\t", index=False)

In [ ]:
# Para no esperar tanto tiempo, descargamos el archivo ya procesado
!wget https://github.com/jorgeortizfuentes/sicss-text-analysis2025/raw/refs/heads/main/noticias_procesadas.tsv

df_filtrado = pd.read_csv("noticias_procesadas.tsv", sep="\t")

df_filtrado

## Entrenando un clasificador

El uso de IA generativa es muy útil para el análisis de textos. Sin embargo, es caro y no genera resultados consistentes. 

El enfoque clásico para entrenar un clasificador de texto requiere de datos etiquetados. Por lo tanto, podemos aprovechar la IA generativa para generar datos etiquetados y luego entrenar un clasificador de texto.

En esta ocasión utilizaremos los embeddings de ´paraphrase-multilingual-MiniLM-L12-v2 para convertir los textos en vectores y luego entrenaremos un modelo utilizando scikit-learn.

In [ ]:
!pip install sentence-transformers
!pip install scikit-learn

In [ ]:
from sentence_transformers import SentenceTransformer

sentences = ["Esto es un texto de ejemplo"]

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

embeddings = model.encode(sentences)

print(embeddings)

In [ ]:
len(embeddings[0])

In [ ]:
# Vamos a leer el archivo guardado
df_filtrado = pd.read_csv("noticias_procesadas.tsv", sep="\t")
df_filtrado["genera_temor"]

In [154]:
# Vectorizar el df_filtrado con el modelo SentenceTransformer

df_filtrado["embeddings"] = df_filtrado["texto"].apply(lambda x: model.encode([x])[0])

In [ ]:
# Entrenar modelo de random forest para clasificar los textos (con genera_temor y embeddings)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

rfc_model = RandomForestClassifier(random_state=0)

# Dividir dataset
X = df_filtrado["embeddings"].tolist()
y = df_filtrado["genera_temor"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entrenar modelo
rfc_model.fit(X_train, y_train)

# Predecir
y_pred = rfc_model.predict(X_test)

# Mostrar reporte de clasificación
print(classification_report(y_test, y_pred))

In [ ]:
!pip install xgboost

In [ ]:
# Entrenamos ahora con xgboost
from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=0)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

## Material complementario 

* [Speech and Language Processing (Jurafsky and Martin)](https://web.stanford.edu/~jurafsky/slp3/)
* [spaCy 101](https://spacy.io/usage/spacy-101)
* [LangChain Tutorials](https://python.langchain.com/docs/tutorials/)
* [Spanish NLP library](https://github.com/jorgeortizfuentes/spanish_nlp) 